# Table of Contents
 <p><div class="lev1"><a href="#Accuracy-Calculation-COST-323"><span class="toc-item-num">1&nbsp;&nbsp;</span>Accuracy Calculation COST 323</a></div><div class="lev1"><a href="#References"><span class="toc-item-num">2&nbsp;&nbsp;</span>References</a></div>

# Accuracy Calculation COST 323

The accuracy calculation can be consulting in \cite{jacob1998european}.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 'A', 'B+', 'B', 'C', 'D+', 'D', 'E'
tolerance_table = pd.DataFrame(
    [], columns=('A', 'B+', 'B', 'C', 'D+', 'D', 'E'),
    index=(
        'gwv', 'load_group_axles', 'load_single_axle', 'load_axle_of_group'
    )
)
tolerance_table.loc['gwv',:] = [5, 7, 10, 15, 20, 25, np.inf]
tolerance_table.loc['load_group_axles',:] = [7, 10, 13, 18, 23, 28, np.inf]
tolerance_table.loc['load_single_axle',:] = [8, 11, 15, 20, 25, 30, np.inf]
tolerance_table.loc['load_axle_of_group',:] = [10, 14, 20, 25, 30, 35, np.inf]
tolerance_table

,A,B+,B,C,D+,D,E
gwv,5,7,10,15,20,25,inf
load_group_axles,7,10,13,18,23,28,inf
load_single_axle,8,11,15,20,25,30,inf
load_axle_of_group,10,14,20,25,30,35,inf


In [3]:
# conditions 
"""
Test plan
"r1"=full repeatability
"r2"=extended repeatability
"rr1"=limited reproducibility
"rr2"=full reproducibility

Env
"I"=environmental repeatability
"II"=environmental limited reproducibility
"III"=environmental full reproducibility
"""

'\nTest plan\n"r1"=full repeatability\n"r2"=extended repeatability\n"rr1"=limited reproducibility\n"rr2"=full reproducibility\n\nEnv\n"I"=environmental repeatability\n"II"=environmental limited reproducibility\n"III"=environmental full reproducibility\n'

# References

(<a id="cit-jacob1998european" href="#call-jacob1998european">JACOB and O'Brien, 1998</a>) B. JACOB and E.J. O'Brien, ``_European Specification on Weigh-In-Motion of Road Vehicles (COST323)_'', SECOND EUROPEAN CONFERENCE ON WEIGH-IN-MOTION OF ROAD VEHICLES, HELD LISBON, PORTUGAL 14-16 SEPTEMBER 1998,  1998.

